<h2>**Histopathologic Cancer Detection**




github url:  https://github.com/Bessonica/NNprojects

# The problem:
  * we must build СNN for binar clasification. 
  * We need identify metastatic cancer in small image.We must predict a probability that center 32x32px region of a patch contains at least one pixel of tumor tissue. 

# Data:
 * train_labels.csv
 * test and train folders with images

 
# Each sample of data consists of:
    * size 96px x 96px
    * each sample is linked to a class (0 or 1)






# Import all neccesary packages

In [ ]:
import numpy as np
import pandas as pd
import os

#graph packages
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow import keras
from tensorflow.keras.layers import * 
from tensorflow.keras.preprocessing.image import ImageDataGenerator


# Read all data from train_labels.csv

In [ ]:
train_data_full = pd.read_csv('../input/histopathologic-cancer-detection/train_labels.csv', dtype=str)
print("Shape of dataset: ",train_data_full.shape)

train_data_full.id = train_data_full.id + '.tif'
train_data_full.head()

# Is data balanced, lets check it


In [ ]:
(train_data_full.label.value_counts() / len(train_data_full)).to_frame().sort_index().T

# as we can see, data s balanced

# Visualise data

In [ ]:
print("Dataset image example")
h_path = '../input/histopathologic-cancer-detection/train'
sample = train_data_full.sample(n=16).reset_index()

plt.figure(figsize=(6,6))

print("show examples of data")

for i, row in sample.iterrows():
    img = mpimg.imread(f'../input/histopathologic-cancer-detection/train/{row.id}')
    label = row.label
    plt.subplot(4,4,i+1)
    plt.imshow(img)
    plt.text(0,-5,f'label {label}', color='k')

    plt.axis('off')

plt.tight_layout()
plt.show()




# Setup
 * Create train and valid set 
 * Create data loaders
 * setup batch_size variable

In [ ]:
print("START dataloaders")


train_data, valid_data = train_test_split(train_data_full, test_size=0.2, random_state=1, stratify=train_data_full.label)

# Data loaders
train_data_gen = ImageDataGenerator(rescale=1/255)
validation_data_gen = ImageDataGenerator(rescale=1/255)

BATCH_SIZE = 64

train_loader = train_data_gen.flow_from_dataframe(
    dataframe = train_data,
    directory = h_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (96,96)
)

valid_loader = train_data_gen.flow_from_dataframe(
    dataframe = valid_data,
    directory = h_path,
    x_col = 'id',
    y_col = 'label',
    batch_size = BATCH_SIZE,
    seed = 1,
    shuffle = True,
    class_mode = 'categorical',
    target_size = (96,96)
)

print("FINISHED dataloaders")

# Model architecture   

**Decide on CNN architecture**

 * Input shape: 96x96x3 (96 pixel on 96 pixel, with deapth 3(rgb))


  * Feature extractor:
     *  Use 2 layers liike this:
     *  conv2d layer: filters = 64, kernel size = 3x3, activ F = relu
     *  maxPool(2x2): to summerize features made in previous layer
     *  dropOut and batch normalization: to combat overfitting and normalize layers input


   * flatten data

   * clasifier:
     * dense layer: 64 units , activFunction = relu
     * dense layer: 8 units, activFunction = relu
     * batch normalization
     * dense layer(final, output layer): 2 units(2 have 2 optons )  activFunction = sigmoid(not relu because sigm is good for binary classification)


In [ ]:

print("build cnn")
np.random.seed(1)
tf.random.set_seed(1)

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 
cnn = Sequential([
    Conv2D(64, (3,3), activation = 'relu', padding = 'same', input_shape=(96,96,3)),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(64, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),
    
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    Conv2D(128, (3,3), activation = 'relu', padding = 'same'),
    MaxPooling2D(2,2),
    Dropout(0.5),
    BatchNormalization(),

    Flatten(),
    
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(8, activation='relu'),
    Dropout(0.5),
    BatchNormalization(),
    Dense(2, activation='sigmoid')
])
print("ENDED building cnn")
cnn.summary()

# Decide on optimizers parametrs

In [ ]:
# default learn rate is 0.01 which is to high
opt = tf.keras.optimizers.Adam(learning_rate=0.0001)
cnn.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy', tf.keras.metrics.AUC()])

# lets start training
  * epochs = 40

In [ ]:
%%time 

print("start training")

h1 = cnn.fit(
    x = train_loader, 
    steps_per_epoch = len(train_loader), 
    epochs = 40,
    validation_data = valid_loader, 
    validation_steps = len(valid_loader), 
    verbose = 1
)

print("end of training")


#  Inspect (Print history)

In [ ]:
#how history works?
print("history")

history = h1.history
print(history.keys())

# Visualize (plot results)

In [ ]:
#look how more can you change it?
print("start ploting")

def show_res():
    epoch_range = range(1, len(history['loss'])+1)

    plt.figure(figsize=[14,4])
    plt.subplot(1,3,1)
    plt.plot(epoch_range, history['loss'], label='Training')
    plt.plot(epoch_range, history['val_loss'], label='Validation')
    plt.xlabel('Epoch'); plt.ylabel('Loss'); plt.title('Loss')
    plt.legend()
    plt.subplot(1,3,2)
    plt.plot(epoch_range, history['accuracy'], label='Training')
    plt.plot(epoch_range, history['val_accuracy'], label='Validation')
    plt.xlabel('Epoch'); plt.ylabel('Accuracy'); plt.title('Accuracy')
    plt.legend()
    plt.subplot(1,3,3)
    plt.plot(epoch_range, history['auc_1'], label='Training')
    plt.plot(epoch_range, history['val_auc_1'], label='Validation')
    plt.xlabel('Epoch'); plt.ylabel('AUC'); plt.title('AUC')
    plt.legend()
    plt.tight_layout()
    plt.show()
    
show_res()

# Submit data

In [ ]:
test_data = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')
print('Test data has ', test_data.shape, ' size.')
test_data['file_n'] = test_data.id + '.tif'
test_data.head()

# Set up data loaders and batch size

In [ ]:
BATCH_SIZE = 64

test_data_gen = ImageDataGenerator(rescale=1/255)

test_loader = test_data_gen.flow_from_dataframe(
    dataframe = test_data,
    directory = "../input/histopathologic-cancer-detection/test",
    x_col = 'file_n',
    batch_size = BATCH_SIZE,
    shuffle = False,
    class_mode = None,
    target_size = (96,96)
)

# Prediction

In [ ]:
test_end = cnn.predict(test_loader)
print(test_end.shape)

# Take result and save it into submission.csv

In [ ]:
submission = pd.read_csv('../input/histopathologic-cancer-detection/sample_submission.csv')
submission.head()

submission.label = test_end[:,1]
submission.head()

submission.to_csv('submission.csv', header=True, index=False)